In [2]:
from langchain_community.llms import Ollama
from bs4 import BeautifulSoup
import requests

In [3]:
def remove_tags(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [4]:
def getNumberOfSectionsPerChapter(chapterNumber):
  chapterUrl = "https://uscode.house.gov/view.xhtml?req=granuleid%3AUSC-prelim-title18-chapter" + str(chapterNumber) + "&saved=L3ByZWxpbUB0aXRsZTE4L3BhcnQx%7CZ3JhbnVsZWlkOlVTQy1wcmVsaW0tdGl0bGUxOC1wYXJ0MQ%3D%3D%7C%7C%7C0%7Cfalse%7Cprelim&edition=prelim"
  page = requests.get(chapterUrl)
  soup = BeautifulSoup(page.content, "html.parser")
  titleTags = soup.find_all("h3", class_="section-head")
  return len(titleTags)
  


def getAllSectionHeaders():
  sectionTitles = []
  url = "https://uscode.house.gov/view.xhtml?path=/prelim@title18/part1&edition=prelim"
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  titleTags = soup.find_all("h3", class_="section-head")
  for title in titleTags:
    sectionTitles.append(title.text)
  return sectionTitles 

In [5]:
#this method basically gets all the content from a part
def getPart():
   url = "https://uscode.house.gov/view.xhtml?hl=false&edition=prelim&req=granuleid%3AUSC-prelim-title18-chapter1&num=0&saved=L3ByZWxpbUB0aXRsZTE4L3BhcnQx%7CZ3JhbnVsZWlkOlVTQy1wcmVsaW0tdGl0bGUxOC1wYXJ0MQ%3D%3D%7C%7C%7C0%7Cfalse%7Cprelim"
   page = requests.get(url)
   pageText = remove_tags(page.content)
   return pageText
  #  f = open("part1Content.txt", "w")
  #  f.write(pageText)
  #  f.close()

In [6]:
def getSectionInfo(sectionStart, sectionEnd, legalSections):
  for sec in range(sectionStart, sectionEnd+1):
    sectionURL = "https://uscode.house.gov/view.xhtml?hl=false&edition=prelim&req=granuleid%3AUSC-prelim-title18-section" + str(sec) + "&num=0&saved=%7CZ3JhbnVsZWlkOlVTQy1wcmVsaW0tdGl0bGUxOC1zZWN0aW9uNw%3D%3D%7C%7C%7C0%7Cfalse%7Cprelim"
    page = requests.get(sectionURL)
    soup = BeautifulSoup(page.content, "html.parser")
    sectionTitle = soup.title.string
    if ("Repealed" not in sectionTitle):
      legalSectionTitle = "Section " + str(sec) + ": " + sectionTitle
      sectionDescription = soup.find_all("p", class_='statutory-body')
      legalSectionDescription = "Section " + str(sec) + ": "
      for item in sectionDescription:
        legalSectionDescription += item.text
        legalSectionDescription += "\n"
      legalSections[legalSectionTitle] = legalSectionDescription
    else:
      continue
  print("retrieved all section info")

In [58]:
import google.generativeai as genai
import os
from dotenv import load_dotenv, dotenv_values 

def getAllSectionHeaders():
  sectionTitles = []
  url = "https://uscode.house.gov/view.xhtml?path=/prelim@title18/part1&edition=prelim"
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  titleTags = soup.find_all("h3", class_="section-head")
  for title in titleTags:
    sectionTitles.append(title.text)
  return sectionTitles 

def generateGeminiResponse(extractedText):
    load_dotenv()
    genai.configure(api_key = os.getenv("GEMINI_API_KEY"))
    prompt = f"""
      Text:
      {extractedText}

      Instruction:
      For each section from 1 to 27 provided in the text, do the following:
      1. Generate a question based only on the section's content.
      2. Provide the answer to the question, based strictly on the section's information.
      3. Format the response as a Python list, where each item is a dictionary with:
        - A key "input" containing the generated question.
        - A key "output" containing the corresponding answer.

      Ensure that every section is covered and no information is drawn from outside the provided text. Please do not provide any other information other than just the list (no other text).
      STRICTLY RETURN ONLY THE LIST WITH DICTIONARIES AND NOTHING ELSE.
      
      
      Example output:
      [
          {{
              "input": "What is the maximum penalty for being an accessory after the fact under Section 3?",
              "output": "An accessory after the fact can be imprisoned for up to half of the principal's punishment, or up to 15 years if the principal is punishable by life imprisonment or death."
          }},
          {{
              "input": "What is defined as 'United States' in Section 5 of Title 18?",
              "output": "The term 'United States' includes all places and waters subject to U.S. jurisdiction, except the Canal Zone."
          }}
      ]
      """

    model = genai.GenerativeModel("gemini-1.5-flash")
    llmResponse = model.generate_content(prompt)
    return llmResponse

In [8]:
import torch
print(torch.backends.mps.is_available())

True


In [9]:
import ast 

if torch.backends.mps.is_available():
    print("mps used")
    mps_device = torch.device("mps")
else:
    mps_device = torch.device("cpu")



numberOfSectionsInChapter = getNumberOfSectionsPerChapter(1)
i = 1 
j = i + 2
instructionOutputDataset = []

while (j < 5):
    legalSections = {}
    legalSectionsContent = ""
    getSectionInfo(i, j, legalSections)
    for title, content in list(legalSections.items()):
        textContent = f"Title: {title}\nContent: {content}\n"
        legalSectionsContent += textContent
    llmResponse = generateGeminiResponse(legalSectionsContent)
    print(llmResponse._result)
    # llmResponse = ast.literal_eval(llmResponse)
    # instructionOutputDataset.append(llmResponse)
    # i = j 
    # j += 2

mps used
retrieved all section info


DefaultCredentialsError: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.

In [59]:
import ast 

if torch.backends.mps.is_available():
    print("mps used")
    mps_device = torch.device("mps")
else:
    mps_device = torch.device("cpu")
    
legalSections = {}
legalSectionsContent = ""
getSectionInfo(1, 27, legalSections)
for title, content in list(legalSections.items()):
    textContent = f"Title: {title}\nContent: {content}\n"
    legalSectionsContent += textContent
llmResponse = generateGeminiResponse(legalSectionsContent)

mps used


Python-dotenv could not parse statement starting at line 1


retrieved all section info


In [60]:
responseText = llmResponse.text
responseText = ast.literal_eval(responseText)

In [61]:
responseText

[{'input': 'Who can be considered a principal in a crime against the United States according to Section 2?',
  'output': 'A principal is anyone who commits the offense, or aids, abets, counsels, commands, induces, or procures its commission.'},
 {'input': 'What is the punishment for being an accessory after the fact to a crime against the United States?',
  'output': 'An accessory after the fact can be imprisoned for not more than half the maximum term of imprisonment or fined not more than half the maximum fine prescribed for the principal, or both. If the principal is punishable by life imprisonment or death, the accessory shall be imprisoned not more than 15 years.'},
 {'input': 'What is the penalty for misprision of felony under Section 4?',
  'output': 'A person who knows about a felony and conceals it can be fined or imprisoned for up to three years, or both.'},
 {'input': "What does the term 'United States' refer to in a territorial sense according to Section 5?",
  'output': "T

In [31]:
pip install lamini typing-extensions==4.5.0

INFO: pip is looking at multiple versions of azure-storage-blob to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of azure-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of azure-core to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 605.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 1.2 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 69.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12

In [62]:
from lamini import Lamini
load_dotenv()
llm = Lamini(model_name='meta-llama/Meta-Llama-3.1-8B-Instruct',api_key= os.getenv("LAMINI_API_KEY"))
data = responseText
llm.tune(data_or_dataset_id=data)

Python-dotenv could not parse statement starting at line 1


Data pairs uploaded to local.

Your dataset id is: 8ac3fe84ef1b5e9ae3b51709f8cc166129b1813257d56723c9c490252982c466 . Consider using this in the future to train using the same data. 
Eg: llm.train(data_or_dataset_id='8ac3fe84ef1b5e9ae3b51709f8cc166129b1813257d56723c9c490252982c466')
Tuning job submitted! Check status of job 11573 here: https://api.lamini.ai/train/11573


{'job_id': 11573,
 'status': 'CREATED',
 'dataset_id': '8ac3fe84ef1b5e9ae3b51709f8cc166129b1813257d56723c9c490252982c466'}

In [50]:
llm.generate("what is title 18?")

' Title 18 of the United States Code is a federal law that deals with crimes and criminal procedure. It is one of the 27 titles of the United States Code, which is a compilation of federal laws. Title 18 is often referred to as the "Crimes and Criminal Procedure" title.\nTitle 18 includes a wide range of laws, including:\nCrimes against the United States, such as treason, espionage, and sedition\nCrimes against individuals, such as murder, assault, and kidnapping\nCrimes against property, such as theft, burglary, and arson\nCrimes related to the use of firearms and explosives\nCrimes related to the use of computers and the internet\nCrimes related to the transportation of people and goods\nCrimes related to the financial system, such as bank robbery and money laundering\nCrimes related to the environment, such as pollution and wildlife trafficking\nCrimes related to the government, such as bribery and corruption\nThe laws in Title 18 are enforced by federal agencies, such as the Federa